Alan Patricio González Bernal - A01067546
Entrega Mid-Term de modelo Wa-Tor


Introducción
Wa-Tor es una simulación de peces contra tiburones en un mundo toroidal. Fue diseñado por AK Dewdney en 1984 y publicado en Scientific American bajo el nombre “Tiburones y peces libran una guerra ecológica en el planeta toroidal Wa-Tor”.

Tu reto será repetir esta simulación. Así de sencillo.

Descripción
El mundo de Wa-Tor consiste en una cuadrícula bidimensional de celdas que pueden estar vacías, contener un pez o contener un tiburón. La cuadrícula se envuelve de arriba a abajo y de izquierda a derecha, por lo que también se puede considerar como un toroide (de ahí Wa-Tor).

El tiempo avanza en pasos discretos, llamados cronones. En cada cronón, el estado de cada criatura evoluciona de acuerdo con las siguientes reglas:

Todas las criaturas empiezan en una celda vacía;
Cada pez se mueve aleatoriamente a una celda adyacente (usando un vecindario de von Neumann de celdas vecinas directamente al norte, sur, este y oeste); si las celdas están todas ocupadas, no se mueve;
Después de que un pez ha sobrevivido un número fijo de cronones (fertility_threshold), se reproduce dejando otro pez atrás en su celda anterior después de moverse. Posteriormente, su fertilidad se pone a cero;
Después de cada cronón, cada pez pierde una unidad de energía. Si la energía de un pez llega a cero, muere;
Cada tiburón se mueve aleatoriamente a una celda adyacente (usando un vecindario de von Neumann de celdas vecinas directamente al norte, sur, este y oeste) ocupada por un pez y gana una cierta cantidad de energía al hacerlo (al "comerse" el pez); si las celdas adyacentes están todas vacías, se mueve a una de ellas al azar; si las celdas adyacentes están llenas de tiburones, no se mueve;
Después de cada cronón, cada tiburón pierde una unidad de energía. Si la energía de un tiburón llega a cero, muere;
Una vez que un tiburón alcanza su propio umbral de fertilidad, se reproduce de la misma manera que los peces.
Parámetros de la simulación
Wa-Tor está controlado por los siguientes parámetros:

Tamaño del mundo: 75 x 50.
Cantidad de peces: 120
Cantidad de tiburones: 40
Energía inicial de los peces: 20
Energía inicial de los tiburones: 3
Umbral de fertilidad de los peces (fertility_threshold): 4
Umbral de fertilidad de los tiburones (fertility_threshold): 12

In [85]:
# Se generan los imports, los mismos que el segregation model
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con 'Randomactivation'', hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

from matplotlib.animation import PillowWriter


In [86]:
class FishAgent(Agent):
    def __init__(self, current_id, model, energy, fertility = 0):
        super().__init__(current_id, model)
        self.energy = energy
        self.fertility = fertility
        
    def step(self):
        self.advance()
        
    def move(self):
        # Vamos a definir los movimientos posibles para que concuerde con lo del vecindario de Vonn Newmann.
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        
        # Vamos a eliminar de possible moves los espacios utilizados
        empty_cells = [cell for cell in possible_moves if self.model.grid.is_cell_empty(cell)]
        
        # Ahora si, hacemos el movimiento aleatorio.
        if empty_cells:
            new_position = self.random.choice(empty_cells)
            self.model.grid.move_agent(self, new_position)
            
        else:
            return  None
        
    def advance(self):
        # Primero nos movemos
        self.move()
        # Luego, disminuimos la energía
        self.energy -= 1
        # Si la energía es menor o igual a cero, el agente muere
        if self.energy <= 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
        else:
            # Si no, aumentamos la fertilidad
            if self.fertility == 4:
                self.reproduce()
            else:
                self.fertility += 1
    
    def reproduce(self, fertility_threshold = 4):
        if self.fertility == fertility_threshold:
            # Recordar la posición actual del agente padre
            current_position = self.pos
            
            # Obtener las celdas vacías
            empty_cells = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
            empty_cells = [cell for cell in empty_cells if self.model.grid.is_cell_empty(cell)]
            
            # Si hay celdas vacías, reproducirse
            if empty_cells:
                new_position = self.random.choice(empty_cells)
                self.model.grid.move_agent(self, new_position)
                # La energía del hijo es 20
                energy_of_offspring = 20
                # Creamos el hijo
                offspring = FishAgent(self.model.next_id(), self.model, energy=energy_of_offspring, fertility=0)
                self.model.grid.place_agent(offspring, current_position)  # Colocar el offspring en la posición anterior del padre
                self.model.schedule.add(offspring)
                # Reiniciamos la fertilidad del padre
                self.fertility = 0
                
            else:
                return  None
        else:
            return  None

            
class SharkAgent(Agent):
    def __init__(self, current_id, model, energy, fertility = 0):
        super().__init__(current_id, model)
        self.energy = energy
        self.fertility = fertility
        
    def step(self):
        self.advance()
    
    def move(self):
        # Vamos a definir los movimientos posibles para que concuerde con lo del vecindario de Vonn Newmann.
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        # Vamos a eliminar de possible moves los espacios utilizados
        empty_cells = [cell for cell in possible_moves if self.model.grid.is_cell_empty(cell)]
        
        # Ahora si, hacemos el movimiento aleatorio.
        if empty_cells:
            new_position = self.random.choice(empty_cells)
            self.model.grid.move_agent(self, new_position)
            
        else:
            return None
            
    def advance(self):
        
        self.energy -= 1
        if self.energy <= 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
        else:
            self.eat()
            self.move()
            if self.fertility == 12:
                self.reproduce()
            else:
                self.fertility += 1
                
    def reproduce(self, fertility_threshold = 12):
        if self.fertility == fertility_threshold:
            # Recordar la posición actual del agente padre
            current_position = self.pos
            
            empty_cells = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
            empty_cells = [cell for cell in empty_cells if self.model.grid.is_cell_empty(cell)]
            
            if empty_cells:
                new_position = self.random.choice(empty_cells)
                self.model.grid.move_agent(self, new_position)
                energy_of_offspring = 3
                offspring = SharkAgent(self.model.next_id(), self.model, energy=energy_of_offspring, fertility=0)
                self.model.grid.place_agent(offspring, current_position)  # Colocar el offspring en la posición anterior del padre
                self.model.schedule.add(offspring)
                
                self.fertility = 0


    
    def eat(self):
        # Buscamos los vecinos de la celda actual
        cellmates = [n for  n in self.model.grid.get_neighbors(self.pos, moore = False, include_center = False)]
        # Buscamos los peces
        fish = [obj for obj in cellmates if isinstance(obj, FishAgent)]
        # Si hay peces, comemos uno
        if len(fish) > 0:
            prey = self.random.choice(fish)
            # Obtenemos la posición del pez
            (x,y) = prey.pos
            # Eliminamos al pez de la grilla y del schedule
            self.model.grid.remove_agent(prey)
            self.model.schedule.remove(prey)
            # Nos movemos a la posición del pez
            self.model.grid.move_agent(self, (x,y))
            self.energy += 2
        

In [87]:
def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height, 3))  # Usamos una matriz de 3 dimensiones para almacenar colores
        
    # Recorremos la grilla
    for cell in model.grid.coord_iter():
        cell_content, (x, y) = cell
        if cell_content is not None:
            cell_agent = cell_content
            if isinstance(cell_agent, FishAgent):
                grid[x][y] = [1, 0, 0]  # Rojo para los peces
            elif isinstance(cell_agent, SharkAgent):
                grid[x][y] = [0, 0, 1]  # Azul para los tiburones
        else:
            grid[x][y] = [1, 1, 1]  # Blanco para los espacios vacíos
                    
    return grid

In [88]:
class WaTorModel(Model):
    def __init__(self, width, height, initial_fish, initial_sharks):
        self.num_agents = initial_fish + initial_sharks
        self.grid = SingleGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)  
        self.datacollector = DataCollector(model_reporters={"Grid": get_grid})
        self.initial_fish = initial_fish
        self.initial_sharks = initial_sharks
        self.current_id = 0  
            
        num_fish = initial_fish
        while self.grid.exists_empty_cells() and num_fish > 0:
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            if self.grid.is_cell_empty((x, y)):
                energy_of_fish = 20
                fertility_of_fish = 0
                fish = FishAgent(self.next_id(), self, energy=energy_of_fish, fertility=fertility_of_fish)
                self.grid.place_agent(fish, (x, y))
                self.schedule.add(fish)
                num_fish -= 1
        num_sharks = initial_sharks 
        while self.grid.exists_empty_cells() and num_sharks > 0:
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            if self.grid.is_cell_empty((x, y)):
                energy_of_shark = 3
                fertility_of_shark = 0
                shark = SharkAgent(self.next_id(), self, energy=energy_of_shark, fertility=fertility_of_shark)
                self.grid.place_agent(shark, (x, y))
                self.schedule.add(shark)
                num_sharks -= 1
                    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
    

In [89]:
GRID_WIDTH = 75
GRID_HEIGHT = 50
INITIAL_FISH = 500
INITIAL_SHARKS = 40
MAX_GENERATIONS = 100

    # Crear el modelo
model = WaTorModel(GRID_WIDTH, GRID_HEIGHT, INITIAL_FISH, INITIAL_SHARKS)

    # Ejecutar simulación
for i in range(MAX_GENERATIONS):
    model.step()


In [90]:
all_grid = model.datacollector.get_model_vars_dataframe()

MovieWriter ffmpeg unavailable; using Pillow instead.


In [91]:
colored_grid = get_grid(model)

# Mostrar la grilla usando matplotlib
plt.imshow(colored_grid)

anim = animation.FuncAnimation(plt.gcf(), lambda i: plt.imshow(all_grid.iloc[i][0]), frames=len(all_grid))
anim.save('Wa-Tor_A01067546_Alan Patricio González Bernal.gif', writer='ffmpeg', fps=10)
